In [161]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [162]:
load_dotenv()

host = os.getenv("host_db_ujeeg")
username = os.getenv("username_db_ujeeg")
password = os.getenv("password_db_ujeeg")
database = os.getenv("database_db_ujeeg")

In [163]:
# get connection
def get_conn_from_database(username, password, host, database):
    url =f'postgresql://{username}:{password}@{host}/{database}?sslmode=require'
    engine = create_engine(url)
    return engine

# read data
def get_data_csv(conn, query):
        df = pd.read_sql(query, conn)
        return df

In [164]:
conn = get_conn_from_database(username, password, host, database)

In [165]:
conn.connect()

# QUESTION SET 1

#### 1. top produk berdasarkan total unit terjual


In [166]:
query_1 = """
select * from cart_items as ci
inner join products pd
ON ci.product_id = pd.product_id 
"""
df_1 = get_data_csv(conn, query_1)
df_1.head()

,cart_item_id,cart_id,product_id,order_amt,product_id,name,price,description,category_id,image_url
0,1,1,66,3.0,66,Frozen Vegetable Dumplings,70000.0,"Dumplings filled with vegetables or tofu, froz...",2,https://fake.pacmann.io/img/66.jpg
1,2,1,97,3.0,97,Congee,46500.0,"A Chinese rice porridge, often served as a bre...",6,https://fake.pacmann.io/img/97.jpg
2,3,1,133,3.0,133,Cilantro Lime Rice,60500.0,A simple rice dish made by mixing cooked rice ...,6,https://fake.pacmann.io/img/133.jpg
3,4,2,60,1.0,60,Frozen Pizza Rolls,42000.0,Miniature pizza snacks filled with cheese and ...,2,https://fake.pacmann.io/img/60.jpg
4,5,2,79,2.0,79,Cortado,33500.0,A coffee drink consisting of equal parts espre...,4,https://fake.pacmann.io/img/79.jpg


In [167]:
df_1 = df_1[["product_id", "name","order_amt"]]
df_agg_1 = df_1.groupby(["name"]).agg({"order_amt":"sum"}).sort_values("order_amt", ascending=False).reset_index()
df_agg_1.columns = ["product_name", "total_order"]
df_1_result  = df_agg_1.head()
df_1_result

,product_name,total_order
0,Biryani,2443.0
1,Congee,2392.0
2,Mee Goreng,1504.0
3,Duck Confit,1457.0
4,Frozen Calzones,1251.0


#### 2. customer yang paling sering beli top produk

In [168]:
query_2 = """
select pd.product_id, pd.name as product_name, ci.order_amt, cs.cust_id, cs.name as customer_name from cart_items as ci
inner join products pd
ON ci.product_id = pd.product_id
inner join carts ct
ON ci.cart_id = ct.cart_id
inner join customers cs
ON ct.cust_id = cs.cust_id 
"""
df_2 = get_data_csv(conn, query_2)
df_2.head()

,product_id,product_name,order_amt,cust_id,customer_name
0,66,Frozen Vegetable Dumplings,3.0,3,CV Farida Budiman Tbk
1,97,Congee,3.0,3,CV Farida Budiman Tbk
2,133,Cilantro Lime Rice,3.0,3,CV Farida Budiman Tbk
3,60,Frozen Pizza Rolls,1.0,618,Perum Waskita (Persero) Tbk
4,79,Cortado,2.0,618,Perum Waskita (Persero) Tbk


In [169]:
top_5 = list(df_1_result["product_name"])

In [170]:
df_2_1 = df_2[df_2["product_name"].isin(top_5)]


In [171]:
df_agg = df_2_1.groupby("customer_name").agg({"order_amt" : "sum"}).sort_values("order_amt", ascending=False).reset_index()
df_agg.columns = ["customer_name", "total_pembelian"]

In [172]:
df_2_result = df_agg.head(1)
df_2_result

,customer_name,total_pembelian
0,UD Wibowo (Persero) Tbk,59.0


#### 3. kota asal customer yang paling sering melakukan pembelian atau transaksi


In [173]:
query_3 = """
select pd.product_id, pd.name as product_name, ci.order_amt, cs.cust_id, cs.name as customer_name, cs.city from cart_items as ci
inner join products pd
ON ci.product_id = pd.product_id
inner join carts ct
ON ci.cart_id = ct.cart_id
inner join customers cs
ON ct.cust_id = cs.cust_id 
"""
df_3 = get_data_csv(conn, query_3)
df_3.head()

,product_id,product_name,order_amt,cust_id,customer_name,city
0,66,Frozen Vegetable Dumplings,3.0,3,CV Farida Budiman Tbk,Jakarta Timur
1,97,Congee,3.0,3,CV Farida Budiman Tbk,Jakarta Timur
2,133,Cilantro Lime Rice,3.0,3,CV Farida Budiman Tbk,Jakarta Timur
3,60,Frozen Pizza Rolls,1.0,618,Perum Waskita (Persero) Tbk,Jakarta Barat
4,79,Cortado,2.0,618,Perum Waskita (Persero) Tbk,Jakarta Barat


In [174]:
df_agg_3 = df_3.groupby("city").agg({"order_amt" : "sum"}).sort_values("order_amt", ascending=False).reset_index()
df_3_result = df_agg_3.head(1)
df_3_result

,city,order_amt
0,Jakarta Timur,14645.0


### customer yang melakukan repeat order (transaksi lebih dari 1)


In [175]:
query_4 = """
select ci.cart_item_id, pd.product_id, pd.name as product_name, ci.order_amt, cs.cust_id, cs.name as customer_name 
from cart_items  ci
inner join products pd
ON ci.product_id = pd.product_id
inner join carts ct
ON ci.cart_id = ct.cart_id
inner join customers cs
ON ct.cust_id = cs.cust_id 
"""
df_4 = get_data_csv(conn, query_4)
df_4.head()

,cart_item_id,product_id,product_name,order_amt,cust_id,customer_name
0,1,66,Frozen Vegetable Dumplings,3.0,3,CV Farida Budiman Tbk
1,2,97,Congee,3.0,3,CV Farida Budiman Tbk
2,3,133,Cilantro Lime Rice,3.0,3,CV Farida Budiman Tbk
3,4,60,Frozen Pizza Rolls,1.0,618,Perum Waskita (Persero) Tbk
4,5,79,Cortado,2.0,618,Perum Waskita (Persero) Tbk


In [176]:
df_agg_4 = df_4.groupby(["cust_id","customer_name"]).agg({"cart_item_id" : "count"}).reset_index()

In [177]:
cond = df_agg_4["cart_item_id"] > 1
df_4_result = df_agg_4[cond]
df_4_result

,cust_id,customer_name,cart_item_id
0,1,PT Nuraini Tbk,74
1,2,PT Hariyah,19
2,3,CV Farida Budiman Tbk,12
3,4,Perum Firmansyah (Persero) Tbk,10
4,5,PD Saefullah Lestari Tbk,109
...,...,...,...
927,996,Perum Andriani Yuniar,15
928,997,PD Puspasari (Persero) Tbk,33
929,998,PT Nababan Hutapea,17
930,999,Perum Rahayu,4


#### 5. jumlah transaksi paling tinggi yang harus dibayar customer  per transaksi (1 keranjang)


In [178]:
query_5 = """
select ci.cart_item_id, pd.product_id, pd.name as product_name, ci.order_amt as total_order, pd.price , cs.cust_id, cs.name as customer_name
from cart_items  ci
inner join products pd
ON ci.product_id = pd.product_id
inner join carts ct
ON ci.cart_id = ct.cart_id
inner join customers cs
ON ct.cust_id = cs.cust_id 
"""
df_5 = get_data_csv(conn, query_5)
df_5.head()

,cart_item_id,product_id,product_name,total_order,price,cust_id,customer_name
0,1,66,Frozen Vegetable Dumplings,3.0,70000.0,3,CV Farida Budiman Tbk
1,2,97,Congee,3.0,46500.0,3,CV Farida Budiman Tbk
2,3,133,Cilantro Lime Rice,3.0,60500.0,3,CV Farida Budiman Tbk
3,4,60,Frozen Pizza Rolls,1.0,42000.0,618,Perum Waskita (Persero) Tbk
4,5,79,Cortado,2.0,33500.0,618,Perum Waskita (Persero) Tbk


In [179]:
df_5 = df_5[["cust_id", "customer_name" ,"total_order", "price"]]
df_5["total_price"] = df_5["total_order"] * df_5["price"]

max_nilai = df_5["total_price"].max()
cond = df_5["total_price"] >= max_nilai
df_5[cond].head()

,cust_id,customer_name,total_order,price,total_price
61,494,CV Melani (Persero) Tbk,3.0,118000.0,354000.0
1635,749,CV Suryono Gunawan Tbk,3.0,118000.0,354000.0
1647,759,UD Puspasari Tbk,3.0,118000.0,354000.0
2576,968,CV Sinaga (Persero) Tbk,3.0,118000.0,354000.0
3532,479,UD Safitri Puspasari (Persero) Tbk,3.0,118000.0,354000.0


#### 6. produk yang paling mahal untuk tiap kategori produk


In [180]:
query_6 = """
select ct.name as category_name, prd.name as product_name, prd.price
from products prd
join categories ct
on ct.category_id = prd.category_id
"""
df_6 = get_data_csv(conn, query_6)
df_6.head()

,category_name,product_name,price
0,noodles,Japchae,72500.0
1,frozen,Frozen Tater Tots,35000.0
2,frozen,Frozen Hot Pockets,32000.0
3,chicken_duck,Duck Foie Gras,112500.0
4,noodles,Mee Goreng,66000.0


In [181]:
df_agg = df_6.sort_values("price", ascending=False).reset_index()
df_agg_result = df_agg.drop_duplicates("category_name", keep="first")
df_agg_result

,index,category_name,product_name,price
0,89,chicken_duck,Chicken Biryani,118000.0
23,16,noodles,Laksa,75500.0
31,54,rice,Lemon Rice,70500.0
34,65,frozen,Frozen Vegetable Dumplings,70000.0
97,22,beverages,Pu-erh Tea,39000.0
107,134,coffee,Iced Coffee,35500.0


#### 7. produk favorit untuk tiap kategori produk berdasarkan jumlah transaksi


In [182]:
query_7 = """
select ci.cart_item_id, ct.name as category_name, pd.product_id, pd.name as product_name, ci.order_amt as total_order, pd.price
from cart_items ci
join products pd
ON ci.product_id = pd.product_id
join categories as ct
ON pd.category_id = ct.category_id
"""
df_7 = get_data_csv(conn, query_7)
df_7.head()

,cart_item_id,category_name,product_id,product_name,total_order,price
0,1,frozen,66,Frozen Vegetable Dumplings,3.0,70000.0
1,2,rice,97,Congee,3.0,46500.0
2,3,rice,133,Cilantro Lime Rice,3.0,60500.0
3,4,frozen,60,Frozen Pizza Rolls,1.0,42000.0
4,5,coffee,79,Cortado,2.0,33500.0


In [183]:
df_agg = df_7.groupby(["product_name", "category_name", "product_id"]).agg({"total_order" : "sum"}).sort_values(["total_order", "product_id"], ascending=[False,True]).reset_index()
df_agg_result = df_agg.drop_duplicates("category_name", keep="first")

In [184]:
df_agg_result

,product_name,category_name,product_id,total_order
0,Biryani,rice,24,2443.0
2,Mee Goreng,noodles,5,1504.0
3,Duck Confit,chicken_duck,33,1457.0
4,Frozen Calzones,frozen,119,1251.0
5,Drip Coffee,coffee,65,834.0
11,Orange Juice,beverages,72,606.0


#### 8. produk dengan jumlah tag terbanyak

In [185]:
query_8 = """
select ci.cart_item_id, pd.product_id, pd.name as product_name, tg.tag_id, tg.tag
from cart_items ci
join products pd
ON ci.product_id = pd.product_id
join tags tg
ON pd.product_id = tg.product_id
"""
df_8 = get_data_csv(conn, query_8)
df_8.head()

,cart_item_id,product_id,product_name,tag_id,tag
0,1,66,Frozen Vegetable Dumplings,298,filled
1,1,66,Frozen Vegetable Dumplings,297,frozen
2,1,66,Frozen Vegetable Dumplings,296,tofu
3,1,66,Frozen Vegetable Dumplings,295,dumplings
4,1,66,Frozen Vegetable Dumplings,294,easy


In [186]:
df_agg = df_8.groupby(["product_id", "product_name"]).agg({"tag_id" : "count"}).sort_values(["tag_id"], ascending=False).reset_index()
terbanyak = df_agg["tag_id"].max()
cond = df_agg["tag_id"] == terbanyak
df_result_8 = df_agg[cond]
df_result_8

,product_id,product_name,tag_id
0,24,Biryani,7272


#### 9. top 5 spenders di ecommerce

In [187]:
query_9 = """
select ci.cart_item_id, pd.product_id, pd.name as product_name, cs.cust_id, cs.name as customer_name , ci.order_amt, pd.price
from cart_items  ci
inner join products pd
ON ci.product_id = pd.product_id
inner join carts ct
ON ci.cart_id = ct.cart_id
inner join customers cs
ON ct.cust_id = cs.cust_id 
"""
df_9 = get_data_csv(conn, query_9)
df_9.head()

,cart_item_id,product_id,product_name,cust_id,customer_name,order_amt,price
0,1,66,Frozen Vegetable Dumplings,3,CV Farida Budiman Tbk,3.0,70000.0
1,2,97,Congee,3,CV Farida Budiman Tbk,3.0,46500.0
2,3,133,Cilantro Lime Rice,3,CV Farida Budiman Tbk,3.0,60500.0
3,4,60,Frozen Pizza Rolls,618,Perum Waskita (Persero) Tbk,1.0,42000.0
4,5,79,Cortado,618,Perum Waskita (Persero) Tbk,2.0,33500.0


In [188]:
df_9["total_spend"] = df_9["order_amt"] * df_9["price"]

In [189]:
df_agg = df_9.groupby(["cust_id","customer_name"]).agg({"total_spend" : "sum"}).sort_values("total_spend" , ascending=False).reset_index()
df_agg.head(5)

,cust_id,customer_name,total_spend
0,223,CV Anggriawan,20147000.0
1,780,UD Wibowo (Persero) Tbk,17940500.0
2,807,PT Padmasari,15043500.0
3,54,PD Yolanda Tbk,14219500.0
4,13,Perum Sitorus Tampubolon (Persero) Tbk,14098000.0


#### 10. di bulan apa yang paling banyak untuk melakukan transaksi

In [190]:
query_10 = """
select *
from cart_items ci
join carts ct
using ("cart_id")
"""
df_10 = get_data_csv(conn, query_10)
df_10.head()

,cart_id,cart_item_id,product_id,order_amt,cust_id,trx_date
0,1,1,66,3.0,3,2023-01-01
1,1,2,97,3.0,3,2023-01-01
2,1,3,133,3.0,3,2023-01-01
3,2,4,60,1.0,618,2023-01-01
4,2,5,79,2.0,618,2023-01-01


In [191]:
df_agg = df_10.groupby("trx_date").agg({"cart_item_id" : "count"}).reset_index()
df_agg["date"] = pd.to_datetime(df_agg["trx_date"])
df_agg["month"]= df_agg["date"].dt.month
df_agg = df_agg.groupby("month").agg({"cart_item_id" : "sum"}).sort_values("cart_item_id", ascending=False).reset_index()
df_agg.columns = ["month", "total_transaksi"]
df_agg

,month,total_transaksi
0,10,2770
1,11,2766
2,9,2745
3,8,2710
4,7,2615
5,6,2500
6,5,2485
7,4,2330
8,3,2325
9,1,2228


# Question Set 2

#### 1. tipe campaign apa yang paling sering dilakukan


In [192]:
query_qs2 = """
select ci.campaign_id, ci.campaign_item_id, cg.campaign_type, cg.name as campaign_name, cg.start_date
from campaign_items ci
join campaigns cg
using ("campaign_id")
"""
df_qs2 = get_data_csv(conn, query_qs2)
df_qs2.head()

,campaign_id,campaign_item_id,campaign_type,campaign_name,start_date
0,1,2,Category,Festival Makanan Sehat,2023-01-01
1,1,3,Category,Festival Makanan Sehat,2023-01-01
2,1,6,Category,Festival Makanan Sehat,2023-01-01
3,3,1,Category,Mei Festival Makanan Lokal,2023-05-01
4,3,3,Category,Mei Festival Makanan Lokal,2023-05-01


In [193]:
df_1 = df_qs2

df_result_1 = df_1.groupby("campaign_type").agg({"campaign_id" : "count"}).sort_values("campaign_id", ascending = False).reset_index()
df_result_1.columns = ["campaign_type", "total_campaign"]
df_result_1

,campaign_type,total_campaign
0,Product,39
1,Category,17


#### 2.campaign apa yang memiliki durasi paling lama sebelum campaign berikutnya dimulai

In [194]:
query_qs22 = """
select *
from campaigns cg
"""
df_qs22 = get_data_csv(conn, query_qs22)
df_qs22.head()

,campaign_id,name,campaign_type,start_date
0,1,Festival Makanan Sehat,Category,2023-01-01
1,2,Bulan Cita Rasa Nusantara,Product,2023-02-01
2,3,Mei Festival Makanan Lokal,Category,2023-05-01
3,4,Festival Pedas Juli,Category,2023-07-01
4,5,Bulan Makanan Manis,Category,2023-09-01


In [195]:
df_2 = df_qs22
df_2["end_date"] = df_2["start_date"].shift(-1)
df_2["start_date"] = pd.to_datetime(df_2["start_date"])
df_2["end_date"] = pd.to_datetime(df_2["end_date"])
df_2["duration_time"] =df_2["end_date"] - df_2["start_date"]
max = df_2["duration_time"].max()
cond = df_2["duration_time"] == max
df_2_result = df_2[cond]
df_2_result[["name", "campaign_type", "start_date", "duration_time"]]

,name,campaign_type,start_date,duration_time
5,November Nikmat,Product,2023-11-01,92 days
7,Maret Rasa Pedas,Product,2024-03-01,92 days


#### 3. product apa yang paling sering di campaign pada tipe campaign produk dan tampilkan berapa kali dilakukan campaign


In [196]:
query_qs23= """
select ci.campaign_id, ci.campaign_item_id, cg.campaign_type, ci.campaign_item, cg.name as campaign_name, cg.start_date
from campaign_items ci
join campaigns cg
using ("campaign_id")
"""
df_qs23= get_data_csv(conn, query_qs23)
df_qs23.head()

,campaign_id,campaign_item_id,campaign_type,campaign_item,campaign_name,start_date
0,1,2,Category,frozen,Festival Makanan Sehat,2023-01-01
1,1,3,Category,chicken_duck,Festival Makanan Sehat,2023-01-01
2,1,6,Category,rice,Festival Makanan Sehat,2023-01-01
3,3,1,Category,noodles,Mei Festival Makanan Lokal,2023-05-01
4,3,3,Category,chicken_duck,Mei Festival Makanan Lokal,2023-05-01


In [197]:
df_3 = df_qs23
df_3["campaign_type"] = df_3["campaign_type"].astype(str)
cond = df_3["campaign_type"] == "Product"
df_3= df_3[cond]
df_3 = df_qs23.groupby("campaign_item").agg({"campaign_item_id" : "count"}).reset_index()
cond_1 = df_3["campaign_item_id"] > 1
df_3_result = df_3[cond_1].sort_values("campaign_item_id", ascending=False)
df_3_result.columns = ["priduct_iten", "total_campaign"]
df_3_result

,priduct_iten,total_campaign
36,chicken_duck,5
35,beverages,3
38,frozen,3
13,Flat White,2
26,Jasmine Tea,2
31,Pilaf,2
27,Lasagna,2
37,coffee,2
39,noodles,2
40,rice,2


#### 4. hitung ada berapa banyak unique produk dari masing - masing kategori

In [198]:
df_4 = df_qs23
df_4.head()

,campaign_id,campaign_item_id,campaign_type,campaign_item,campaign_name,start_date
0,1,2,Category,frozen,Festival Makanan Sehat,2023-01-01
1,1,3,Category,chicken_duck,Festival Makanan Sehat,2023-01-01
2,1,6,Category,rice,Festival Makanan Sehat,2023-01-01
3,3,1,Category,noodles,Mei Festival Makanan Lokal,2023-05-01
4,3,3,Category,chicken_duck,Mei Festival Makanan Lokal,2023-05-01


In [199]:
df_agg = df_4.groupby(["campaign_item", "campaign_type"]).agg({"campaign_id":"count"})
df_agg = df_agg.groupby(["campaign_type"]).agg({"campaign_id":"sum"}).reset_index()
df_agg

,campaign_type,campaign_id
0,Category,17
1,Product,39


#### 5. kategori produk favorit untuk tiap bulannya berdasarkan jumlah transaksi

In [226]:
query_10 = """
select *
from cart_items ci
join carts ct
on ci.cart_id = ct.cart_id
join products pd
on pd.product_id = ci.product_id
"""
df_10 = get_data_csv(conn, query_10)
df_10.head()

,cart_item_id,cart_id,product_id,order_amt,cart_id,cust_id,trx_date,product_id,name,price,description,category_id,image_url
0,1,1,66,3.0,1,3,2023-01-01,66,Frozen Vegetable Dumplings,70000.0,"Dumplings filled with vegetables or tofu, froz...",2,https://fake.pacmann.io/img/66.jpg
1,2,1,97,3.0,1,3,2023-01-01,97,Congee,46500.0,"A Chinese rice porridge, often served as a bre...",6,https://fake.pacmann.io/img/97.jpg
2,3,1,133,3.0,1,3,2023-01-01,133,Cilantro Lime Rice,60500.0,A simple rice dish made by mixing cooked rice ...,6,https://fake.pacmann.io/img/133.jpg
3,4,2,60,1.0,2,618,2023-01-01,60,Frozen Pizza Rolls,42000.0,Miniature pizza snacks filled with cheese and ...,2,https://fake.pacmann.io/img/60.jpg
4,5,2,79,2.0,2,618,2023-01-01,79,Cortado,33500.0,A coffee drink consisting of equal parts espre...,4,https://fake.pacmann.io/img/79.jpg


In [241]:
df_agg = df_10.groupby(["name", "trx_date"]).agg({"cart_item_id" : "count"}).reset_index()
df_agg["date"] = pd.to_datetime(df_agg["trx_date"])
df_agg["month"]= df_agg["date"].dt.month

df_agg = df_agg.groupby(["name","month"]).agg({"cart_item_id" : "sum"}).sort_values(["month","cart_item_id"], ascending=[True,False]).reset_index()
df_agg_result = df_agg.drop_duplicates("month", keep="first")
df_agg_result


,name,month,cart_item_id
0,Congee,1,91
152,Biryani,2,83
304,Congee,3,98
456,Congee,4,114
608,Biryani,5,105
760,Biryani,6,109
912,Biryani,7,129
1064,Congee,8,112
1216,Biryani,9,113
1368,Congee,10,128
